# Load Dataset 

In [1]:
import json

import pandas as pd
from pathlib import Path
pd.set_option('max_colwidth',300)
from pprint import pprint

from sklearn.model_selection import train_test_split

In [2]:
!unzip java.zip

Archive:  java.zip
   creating: java/
   creating: java/final/
   creating: java/final/jsonl/
   creating: java/final/jsonl/train/
  inflating: java/final/jsonl/train/java_train_12.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_9.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_3.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_5.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_7.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_1.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_10.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_14.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_0.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_6.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_8.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_15.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_2.jsonl.gz  
  inflating: java/final/jsonl/train/java_train_4.jsonl.gz  
  inflating: java/final/j

In [3]:
# decompress this gzip file
!gzip -d java/final/jsonl/test/java_test_0.jsonl.gz

Read in the file and display the first row.  The data is stored in [JSON Lines](http://jsonlines.org/) format.

In [4]:
with open('java/final/jsonl/test/java_test_0.jsonl', 'r') as f:
    sample_file = f.readlines()
sample_file[0]

'{"repo": "ReactiveX/RxJava", "path": "src/main/java/io/reactivex/internal/observers/QueueDrainObserver.java", "func_name": "QueueDrainObserver.fastPathOrderedEmit", "original_string": "protected final void fastPathOrderedEmit(U value, boolean delayError, Disposable disposable) {\\n        final Observer<? super V> observer = downstream;\\n        final SimplePlainQueue<U> q = queue;\\n\\n        if (wip.get() == 0 && wip.compareAndSet(0, 1)) {\\n            if (q.isEmpty()) {\\n                accept(observer, value);\\n                if (leave(-1) == 0) {\\n                    return;\\n                }\\n            } else {\\n                q.offer(value);\\n            }\\n        } else {\\n            q.offer(value);\\n            if (!enter()) {\\n                return;\\n            }\\n        }\\n        QueueDrainHelper.drainLoop(q, observer, delayError, disposable, this);\\n    }", "language": "java", "code": "protected final void fastPathOrderedEmit(U value, boolean d

We can utilize the fact that each line in the file is valid json, and display the first row in a more human readable form:

In [5]:
pprint(json.loads(sample_file[0]))

{'code': 'protected final void fastPathOrderedEmit(U value, boolean '
         'delayError, Disposable disposable) {\n'
         '        final Observer<? super V> observer = downstream;\n'
         '        final SimplePlainQueue<U> q = queue;\n'
         '\n'
         '        if (wip.get() == 0 && wip.compareAndSet(0, 1)) {\n'
         '            if (q.isEmpty()) {\n'
         '                accept(observer, value);\n'
         '                if (leave(-1) == 0) {\n'
         '                    return;\n'
         '                }\n'
         '            } else {\n'
         '                q.offer(value);\n'
         '            }\n'
         '        } else {\n'
         '            q.offer(value);\n'
         '            if (!enter()) {\n'
         '                return;\n'
         '            }\n'
         '        }\n'
         '        QueueDrainHelper.drainLoop(q, observer, delayError, '
         'disposable, this);\n'
         '    }',
 'code_tokens': ['pr

Definitions of each of the above fields are located in the  in the README.md file in the root of this repository.

In [6]:
java_files = sorted(Path('java').glob('**/*.gz'))

In [7]:
print(f'Total number of files: {len(java_files):,}')

Total number of files: 17


To make analysis of this dataset easier, we can load all of the data into a pandas dataframe: 

In [8]:
columns_long_list = ['repo', 'path', 'url', 'code', 
                     'code_tokens', 'docstring', 'docstring_tokens', 
                     'language', 'partition']

columns_short_list = ['code_tokens', 'docstring_tokens', 
                      'language', 'partition']

def jsonl_list_to_dataframe(file_list, columns=columns_long_list):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f, 
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

This is what the python dataset looks like:

In [9]:
df = jsonl_list_to_dataframe(java_files)

In [10]:
df.head(3)

,repo,path,url,code,code_tokens,docstring,docstring_tokens,language,partition
0,spring-projects/spring-boot,spring-boot-project/spring-boot/src/main/java/org/springframework/boot/context/properties/bind/IndexedElementsBinder.java,https://github.com/spring-projects/spring-boot/blob/0b27f7c70e164b2b1a96477f1d9c1acba56790c1/spring-boot-project/spring-boot/src/main/java/org/springframework/boot/context/properties/bind/IndexedElementsBinder.java#L67-L77,"protected final void bindIndexed(ConfigurationPropertyName name, Bindable<?> target,\n\t\t\tAggregateElementBinder elementBinder, ResolvableType aggregateType,\n\t\t\tResolvableType elementType, IndexedCollectionSupplier result) {\n\t\tfor (ConfigurationPropertySource source : getContext().getSo...","[protected, final, void, bindIndexed, (, ConfigurationPropertyName, name, ,, Bindable, <, ?, >, target, ,, AggregateElementBinder, elementBinder, ,, ResolvableType, aggregateType, ,, ResolvableType, elementType, ,, IndexedCollectionSupplier, result, ), {, for, (, ConfigurationPropertySource, sou...","Bind indexed elements to the supplied collection.\n@param name the name of the property to bind\n@param target the target bindable\n@param elementBinder the binder to use for elements\n@param aggregateType the aggregate type, may be a collection or an array\n@param elementType the element type\n...","[Bind, indexed, elements, to, the, supplied, collection, .]",java,train
1,spring-projects/spring-boot,spring-boot-project/spring-boot/src/main/java/org/springframework/boot/web/servlet/AbstractFilterRegistrationBean.java,https://github.com/spring-projects/spring-boot/blob/0b27f7c70e164b2b1a96477f1d9c1acba56790c1/spring-boot-project/spring-boot/src/main/java/org/springframework/boot/web/servlet/AbstractFilterRegistrationBean.java#L75-L80,"public void setServletRegistrationBeans(\n\t\t\tCollection<? extends ServletRegistrationBean<?>> servletRegistrationBeans) {\n\t\tAssert.notNull(servletRegistrationBeans,\n\t\t\t\t""ServletRegistrationBeans must not be null"");\n\t\tthis.servletRegistrationBeans = new LinkedHashSet<>(servletRegist...","[public, void, setServletRegistrationBeans, (, Collection, <, ?, extends, ServletRegistrationBean, <, ?, >, >, servletRegistrationBeans, ), {, Assert, ., notNull, (, servletRegistrationBeans, ,, ""ServletRegistrationBeans must not be null"", ), ;, this, ., servletRegistrationBeans, =, new, LinkedH...",Set {@link ServletRegistrationBean}s that the filter will be registered against.\n@param servletRegistrationBeans the Servlet registration beans,"[Set, {]",java,train
2,spring-projects/spring-boot,spring-boot-project/spring-boot/src/main/java/org/springframework/boot/web/servlet/AbstractFilterRegistrationBean.java,https://github.com/spring-projects/spring-boot/blob/0b27f7c70e164b2b1a96477f1d9c1acba56790c1/spring-boot-project/spring-boot/src/main/java/org/springframework/boot/web/servlet/AbstractFilterRegistrationBean.java#L98-L103,"public void addServletRegistrationBeans(\n\t\t\tServletRegistrationBean<?>... servletRegistrationBeans) {\n\t\tAssert.notNull(servletRegistrationBeans,\n\t\t\t\t""ServletRegistrationBeans must not be null"");\n\t\tCollections.addAll(this.servletRegistrationBeans, servletRegistrationBeans);\n\t}","[public, void, addServletRegistrationBeans, (, ServletRegistrationBean, <, ?, >, ..., servletRegistrationBeans, ), {, Assert, ., notNull, (, servletRegistrationBeans, ,, ""ServletRegistrationBeans must not be null"", ), ;, Collections, ., addAll, (, this, ., servletRegistrationBeans, ,, servletReg...",Add {@link ServletRegistrationBean}s for the filter.\n@param servletRegistrationBeans the servlet registration beans to add\n@see #setServletRegistrationBeans,"[Add, {]",java,train


In [12]:
df.language.value_counts()

language
java    469779
Name: count, dtype: int64

In [13]:
data = df['code']

In [14]:
len(data)

469779

In [15]:
selected = data[:26000]

In [16]:
train_df, test_df = train_test_split(selected, test_size=0.2, random_state=42)

# Save the splits to CSV files
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)